In [2]:
"""
Nasdaq Portfolio — Binary Markowitz + Partitioning
=================================================

* **Step 1**  Download 10 years of daily prices for (up to) 48 Nasdaq tickers.
* **Step 2**  Denoise the covariance matrix via Marchenko–Pastur (RMT).
* **Step 3**  Spectral‑cluster the cleaned correlation into N communities (3 assets each).
* **Step 4**  **Global MILP**: binary Markowitz (0/1 weights), select N*3 assets with ≥3 per cluster.
* **Step 5**  **Pipeline**: solve N sub‑MILPs (3 picks per cluster) and union the results.
* **Step 6**  Compare objectives → approximation ratio.

The binary Markowitz objective is the same as in the image:

```
min   q · xᵀ Σ x  −  μᵀ x
s.t.  x ∈ {0,1}ⁿ ,  ∑ x = 12 ,  ∑_{i∈cluster_j} x_i ≥ 3
```
"""

'\nNasdaq Portfolio — Binary Markowitz + Partitioning\n=================================================\n\n* **Step\xa01**  Download 10\u202fyears of daily prices for (up to) 48 Nasdaq tickers.\n* **Step\xa02**  Denoise the covariance matrix via Marchenko–Pastur (RMT).\n* **Step\xa03**  Spectral‑cluster the cleaned correlation into *k\xa0=\xa04* communities (≥3\xa0assets each).\n* **Step\xa04**  **Global MILP**: binary Markowitz (0/1 weights), select **exactly 12** assets with ≥3 per cluster.\n* **Step\xa05**  **Pipeline**: solve 4 sub‑MILPs (3 picks per cluster) and union the results.\n* **Step\xa06**  Compare objectives → approximation ratio.\n\nThe binary Markowitz objective is the same as in the image:\n\n```\nmin   q · xᵀ Σ x \xa0−\xa0 μᵀ x\ns.t.  x ∈ {0,1}ⁿ ,  ∑ x = 12 ,  ∑_{i∈cluster_j} x_i ≥ 3\n```\n'

In [3]:
import importlib, subprocess, sys, warnings, math, requests, numpy as np, pandas as pd
warnings.filterwarnings("ignore")

# Install required packages
for pkg in ["yfinance","numpy","pandas","scipy","scikit-learn","pulp","tqdm"]:
    try:
        importlib.import_module(pkg)
    except ImportError:
        subprocess.check_call([sys.executable,"-m","pip","install",pkg,"-q"])

import yfinance as yf
from numpy.linalg import eigh
from sklearn.cluster import SpectralClustering
import pulp as pl
import random
import time
from datetime import datetime, timedelta
from tqdm import tqdm

In [4]:
random.seed(42)
np.random.seed(42)

In [41]:
# Configuration
N_ASSETS           = 150   # number of tickers
CLUSTERS           = 10    # spectral clusters
ASSETS_PER_CLUSTER = 3     # min picks per cluster
PORTFOLIO_SIZE     = CLUSTERS * ASSETS_PER_CLUSTER  # total assets
Q                  = 0.5   # risk‑aversion

LOOKBACK_YEARS     = 10
TRADING_DAYS       = 252
MIN_POINTS         = int(TRADING_DAYS * LOOKBACK_YEARS * 0.95)  # ≥95% of expected rows

print("="*60)
print(f"NASDAQ PORTFOLIO OPTIMIZATION - {N_ASSETS} Assets")
print("="*60)

NASDAQ PORTFOLIO OPTIMIZATION - 150 Assets


In [42]:
# Step 1: Fetch NASDAQ listings
print("\n📊 Fetching NASDAQ listings...")
nasdaq_url = "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt"
nasdaq_df = pd.read_csv(nasdaq_url, sep="|", dtype=str)

# Clean the data
nasdaq_df = nasdaq_df[~nasdaq_df["Symbol"].str.contains("File Creation Time", na=False)]
nasdaq_df = nasdaq_df[nasdaq_df["Test Issue"] != "Y"]

# Filter out non-common stocks
bad_suffixes = ("W", "R", "P", "Q", "Y", "Z")  # warrants, rights, preferred, etc.
symbols = nasdaq_df["Symbol"].dropna().astype(str).str.strip()
tickers = [s for s in symbols if not s.endswith(bad_suffixes) and len(s) <= 5 and s.isalpha()]

print(f"Found {len(tickers)} potential common stocks")

# Shuffle for random selection
random.shuffle(tickers)

# Step 2: Download price data
print(f"\n📥 Downloading {LOOKBACK_YEARS}-year price data for {N_ASSETS} assets...")
print("This may take a few minutes...")

prices_list, valid_tickers = [], []
attempt_count = 0
batch_size = 5  # Download in small batches to avoid rate limits

pbar = tqdm(total=N_ASSETS, desc="Valid assets found")

for i in range(0, len(tickers), batch_size):
    if len(valid_tickers) >= N_ASSETS:
        break

    batch = tickers[i:i+batch_size]
    attempt_count += len(batch)

    try:
        # Download batch
        data = yf.download(
            batch,
            period=f"{LOOKBACK_YEARS}y",
            interval="1d",
            auto_adjust=True,
            progress=False,
            threads=False
        )

        # Handle single ticker case
        if len(batch) == 1 and not data.empty:
            close_data = data["Close"] if "Close" in data.columns else data
            if not isinstance(close_data, pd.DataFrame):
                close_data = pd.DataFrame({batch[0]: close_data})
        else:
            close_data = data["Close"] if "Close" in data.columns else data

        # Check each ticker in batch
        for ticker in batch:
            if len(valid_tickers) >= N_ASSETS:
                break

            try:
                if ticker in close_data.columns:
                    ticker_data = close_data[ticker].dropna()

                    # Check if we have enough data points
                    if len(ticker_data) >= MIN_POINTS:
                        ticker_data.name = ticker
                        prices_list.append(ticker_data)
                        valid_tickers.append(ticker)
                        pbar.update(1)
            except:
                continue

    except Exception as e:
        # Rate limit handling
        if "429" in str(e) or "rate" in str(e).lower():
            time.sleep(2)
        continue

    # Small delay between batches
    time.sleep(0.1)

pbar.close()

if len(valid_tickers) < N_ASSETS:
    print(f"\n⚠️  Only found {len(valid_tickers)} assets with {LOOKBACK_YEARS}-year history")
    print("Continuing with available assets...")
    N_ASSETS = len(valid_tickers)
    PORTFOLIO_SIZE = min(PORTFOLIO_SIZE, N_ASSETS)


📊 Fetching NASDAQ listings...
Found 3818 potential common stocks

📥 Downloading 10-year price data for 150 assets...
This may take a few minutes...


Valid assets found: 100%|██████████| 150/150 [01:05<00:00,  2.30it/s]


In [43]:
# Step 3: Build price matrix and calculate returns
print(f"\n📈 Building price matrix for {len(valid_tickers)} assets...")
prices = pd.concat(prices_list, axis=1)
returns = prices.pct_change().dropna()

# Remove any remaining NaN columns
valid_cols = returns.columns[returns.notna().all()]
returns = returns[valid_cols]
prices = prices[valid_cols]
valid_tickers = list(valid_cols)

print(f"Final dataset: {len(valid_tickers)} assets × {len(returns)} days")


📈 Building price matrix for 150 assets...
Final dataset: 150 assets × 2396 days


In [44]:
# Step 4: Calculate statistics
print("\n🧮 Calculating return and risk metrics...")
mu = returns.mean().values * TRADING_DAYS  # Annualized returns
sigma = returns.cov().values * TRADING_DAYS  # Annualized covariance

# Save metrics for reference
metrics_df = pd.DataFrame({
    "Asset": valid_tickers,
    "Annual_Return": mu,
    "Annual_Volatility": np.sqrt(np.diag(sigma)),
    "Sharpe_Ratio": mu / np.sqrt(np.diag(sigma))
}).round(6)

metrics_df.to_csv("portfolio_metrics.csv", index=False)
print("Saved metrics to portfolio_metrics.csv")


🧮 Calculating return and risk metrics...
Saved metrics to portfolio_metrics.csv


In [45]:
# Step 5: RMT Denoising
def rmt_denoise(sigma, n_obs):
    """RMT denoising with stability improvements."""
    n = len(sigma)

    # Ensure symmetric
    sigma = (sigma + sigma.T) / 2

    # Add regularization
    sigma += np.eye(n) * 1e-8

    # Calculate correlation matrix
    std = np.sqrt(np.diag(sigma))
    std = np.maximum(std, 1e-10)
    corr = sigma / np.outer(std, std)
    np.fill_diagonal(corr, 1.0)

    # Eigenvalue decomposition
    vals, vecs = eigh(corr)
    vals, vecs = vals[::-1], vecs[:, ::-1]

    # Marchenko-Pastur bounds
    q = n / n_obs
    if q >= 1:
        q = 0.99

    lmin = (1 - np.sqrt(q)) ** 2
    lmax = (1 + np.sqrt(q)) ** 2

    # Clean eigenvalues
    clean_vals = vals.copy()
    noise_idx = (vals >= lmin) & (vals <= lmax)

    if np.any(noise_idx):
        avg_noise = np.mean(vals[noise_idx])
        clean_vals[noise_idx] = avg_noise

    # Remove market mode if dominant
    if clean_vals[0] > lmax * 2:
        clean_vals[0] = clean_vals[1]

    # Ensure positive
    clean_vals = np.maximum(clean_vals, 1e-8)

    # Reconstruct
    corr_clean = (vecs * clean_vals) @ vecs.T
    corr_clean = (corr_clean + corr_clean.T) / 2
    np.fill_diagonal(corr_clean, 1.0)

    return corr_clean * np.outer(std, std)

print("\n🔧 Applying RMT denoising...")
sigma_clean = rmt_denoise(sigma, len(returns))


🔧 Applying RMT denoising...


In [46]:
# Step 6: Spectral Clustering
def spectral_clusters(corr, k, min_sz):
    """Spectral clustering on correlation matrix."""
    n = len(corr)

    # Affinity matrix
    aff = np.abs(corr) ** 2
    np.fill_diagonal(aff, 1.0)
    aff = (aff + aff.T) / 2

    # Clustering
    try:
        clustering = SpectralClustering(
            n_clusters=k,
            affinity='precomputed',
            random_state=42,
            n_init=10
        )
        labels = clustering.fit_predict(aff)
    except:
        # Fallback to random assignment
        labels = np.random.randint(0, k, size=n)

    clusters = [np.where(labels == i)[0].tolist() for i in range(k)]
    clusters = [c for c in clusters if len(c) > 0]

    # Merge small clusters
    i = 0
    while i < len(clusters):
        if len(clusters[i]) < min_sz and len(clusters) > 1:
            small = clusters.pop(i)
            # Add to random cluster
            if clusters:
                clusters[np.random.randint(len(clusters))].extend(small)
        else:
            i += 1

    # Ensure exactly k clusters
    while len(clusters) < k and any(len(c) >= 2*min_sz for c in clusters):
        # Split largest
        largest_idx = max(range(len(clusters)), key=lambda i: len(clusters[i]))
        large = clusters[largest_idx]
        mid = len(large) // 2
        clusters[largest_idx] = large[:mid]
        clusters.append(large[mid:])

    return clusters[:k]

# Calculate correlation for clustering
std = np.sqrt(np.diag(sigma_clean))
std = np.maximum(std, 1e-10)
corr_clean = sigma_clean / np.outer(std, std)

print(f"\n🎯 Clustering into {CLUSTERS} groups...")
clusters = spectral_clusters(corr_clean, CLUSTERS, ASSETS_PER_CLUSTER)
print("Cluster sizes:", [len(c) for c in clusters])

# Create membership array
membership = np.zeros(len(valid_tickers), dtype=int)
for ci, cluster in enumerate(clusters):
    for idx in cluster:
        if idx < len(valid_tickers):
            membership[idx] = ci


🎯 Clustering into 10 groups...
Cluster sizes: [5, 20, 19, 15, 24, 7, 16, 14, 19, 11]


In [47]:
# Step 7: Binary Optimization
def optimise_binary(mu, sigma, K, q, membership, min_per, time_limit=3000):
    """
    Binary Markowitz optimization with cluster constraints and solver diagnostics.

    Args:
        mu (array): Expected returns (length n).
        sigma (2D array): Covariance matrix (n x n).
        K (int): Number of assets to select.
        q (float): Risk aversion parameter.
        membership (array): Cluster assignment for each asset (length n).
        min_per (int): Minimum assets per cluster.
        time_limit (int): Solver time limit in seconds (default 300).

    Returns:
        selected (list): Indices of selected assets.
        obj_value (float): Objective value.
    """
    n = len(mu)
    assert 1 <= K <= n, f"K must be between 1 and {n} (got {K})"

    # Variables
    x = pl.LpVariable.dicts("x", range(n), 0, 1, pl.LpBinary)
    y = {(i, j): pl.LpVariable(f"y_{i}_{j}", 0, 1, pl.LpBinary)
         for i in range(n) for j in range(i, n)}

    # Problem
    prob = pl.LpProblem("BinMarkowitz", pl.LpMinimize)

    # Linearization constraints
    for (i, j), yij in y.items():
        prob += yij <= x[i]
        prob += yij <= x[j]
        prob += yij >= x[i] + x[j] - 1

    # Objective
    prob += (pl.lpSum(q * sigma[i, j] * y[(i, j)] * (2 if i != j else 1)
                      for (i, j) in y) -
             pl.lpSum(mu[i] * x[i] for i in range(n)))

    # Cardinality constraint
    prob += pl.lpSum(x.values()) == K

    # Cluster constraints
    for c in set(membership):
        idx = [i for i in range(n) if membership[i] == c]
        if len(idx) >= min_per:
            prob += pl.lpSum(x[i] for i in idx) >= min_per

    # Solve with time limit and print solver status
    status = prob.solve(pl.PULP_CBC_CMD(msg=True, timeLimit=time_limit))
    status_str = pl.LpStatus[prob.status]
    print(f"Solver status: {status_str}")

    if status_str != "Optimal":
        print("Warning: Solver did not find an optimal solution. Check constraints and cluster sizes.")

    selected = [i for i in range(n) if x[i] is not None and pl.value(x[i]) > 0.5]
    obj_value = pl.value(prob.objective) if prob.objective is not None else 0

    return selected, obj_value

In [48]:
# Step 8: Global Optimization
print("\n" + "="*60)
print("🌐 GLOBAL OPTIMIZATION")
print("="*60)

try:
    global_idx, global_obj = optimise_binary(
        mu, sigma_clean, PORTFOLIO_SIZE, Q, membership, ASSETS_PER_CLUSTER
    )
    print(f"Objective value: {global_obj:.6f}")
    print(f"Selected {len(global_idx)} assets")
except Exception as e:
    print(f"Optimization failed: {e}")
    # Fallback: select top Sharpe ratio assets
    sharpe = mu / np.sqrt(np.diag(sigma_clean))
    global_idx = np.argsort(sharpe)[-PORTFOLIO_SIZE:].tolist()
    global_obj = 0


🌐 GLOBAL OPTIMIZATION
Solver status: Optimal
Objective value: -2.724654
Selected 30 assets


In [49]:
# Step 9: Pipeline Optimization
print("\n" + "="*60)
print("🔗 PIPELINE OPTIMIZATION")
print("="*60)

pipe_idx = []
for ci, cluster in enumerate(clusters):
    if len(cluster) < ASSETS_PER_CLUSTER:
        continue

    sub_mu = mu[cluster]
    sub_sig = sigma_clean[np.ix_(cluster, cluster)]

    try:
        picks, _ = optimise_binary(
            sub_mu, sub_sig,
            min(ASSETS_PER_CLUSTER, len(cluster)),
            Q * 0.8,  # Slightly lower risk aversion
            [0] * len(cluster),
            min(ASSETS_PER_CLUSTER, len(cluster))
        )
        pipe_idx.extend([cluster[i] for i in picks])
        print(f"Cluster {ci}: selected {len(picks)} assets")
    except:
        # Fallback to top Sharpe in cluster
        sharpe = sub_mu / np.sqrt(np.diag(sub_sig))
        top_k = min(ASSETS_PER_CLUSTER, len(cluster))
        best = np.argsort(sharpe)[-top_k:]
        pipe_idx.extend([cluster[i] for i in best])

# Ensure correct portfolio size
pipe_idx = pipe_idx[:PORTFOLIO_SIZE]

# Calculate pipeline objective
pipe_x = np.zeros(len(mu))
for idx in pipe_idx:
    if idx < len(pipe_x):
        pipe_x[idx] = 1
pipe_obj = Q * pipe_x @ sigma_clean @ pipe_x - mu @ pipe_x


🔗 PIPELINE OPTIMIZATION
Solver status: Optimal
Cluster 0: selected 3 assets
Solver status: Optimal
Cluster 1: selected 3 assets
Solver status: Optimal
Cluster 2: selected 3 assets
Solver status: Optimal
Cluster 3: selected 3 assets
Solver status: Optimal
Cluster 4: selected 3 assets
Solver status: Optimal
Cluster 5: selected 3 assets
Solver status: Optimal
Cluster 6: selected 3 assets
Solver status: Optimal
Cluster 7: selected 3 assets
Solver status: Optimal
Cluster 8: selected 3 assets
Solver status: Optimal
Cluster 9: selected 3 assets


In [50]:
# Step 10: Results
print("\n" + "="*60)
print("📊 RESULTS SUMMARY")
print("="*60)
print(f"Global objective:   {global_obj:.6f}")
print(f"Pipeline objective: {pipe_obj:.6f}")
if global_obj != 0:
    print(f"Approximation ratio: {pipe_obj/global_obj:.6f}")

print(f"\n🏆 GLOBAL PORTFOLIO ({len(global_idx)} assets):")
global_tickers = [valid_tickers[i] for i in global_idx if i < len(valid_tickers)]
print(", ".join(global_tickers[:10]) + ("..." if len(global_tickers) > 10 else ""))

print(f"\n🔗 PIPELINE PORTFOLIO ({len(pipe_idx)} assets):")
pipe_tickers = [valid_tickers[i] for i in pipe_idx if i < len(valid_tickers)]
print(", ".join(pipe_tickers[:10]) + ("..." if len(pipe_tickers) > 10 else ""))

# Save results
results_df = pd.DataFrame({
    "Method": ["Global", "Pipeline"],
    "Objective": [global_obj, pipe_obj],
    "Assets": [len(global_idx), len(pipe_idx)]
})
results_df.to_csv("optimization_results.csv", index=False)

# Save portfolios
pd.DataFrame({"Global_Portfolio": global_tickers}).to_csv("global_portfolio.csv", index=False)
pd.DataFrame({"Pipeline_Portfolio": pipe_tickers}).to_csv("pipeline_portfolio.csv", index=False)

print("\n✅ Results saved to:")
print("   - portfolio_metrics.csv")
print("   - optimization_results.csv")
print("   - global_portfolio.csv")
print("   - pipeline_portfolio.csv")
print("="*60)


📊 RESULTS SUMMARY
Global objective:   -2.724654
Pipeline objective: -2.750189
Approximation ratio: 1.009372

🏆 GLOBAL PORTFOLIO (30 assets):
LFMD, BIS, PUI, MMSI, DXPE, BOKF, VGSH, CHCO, RING, BPOPM...

🔗 PIPELINE PORTFOLIO (30 assets):
PUI, CDL, LVHD, CHCO, TSBK, TFIN, WILC, LINC, CLMB, LFMD...

✅ Results saved to:
   - portfolio_metrics.csv
   - optimization_results.csv
   - global_portfolio.csv
   - pipeline_portfolio.csv
